In [6]:
from kaggle_environments import make
from kaggle_environments.envs.halite.helpers import *

from random import seed
import time

import numpy as np

np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)

In [7]:
NUM_AGENTS = 4
BOARD_SIZE = 7
TURNS = 400

env = make('halite', configuration={"randomSeed": 1, "episodeSteps": TURNS, "size": BOARD_SIZE}, debug=True)
_ = env.reset(num_agents=NUM_AGENTS)

ACTIONS = [
    ShipAction.NORTH,
    ShipAction.EAST,
    ShipAction.SOUTH,
    ShipAction.WEST,
    #ShipAction.CONVERT, #for our toy example we don't want to build new shipyards
    None #None for collecting
]

In [8]:
env.render(mode='ipython')

In [9]:
# This will create an output of
#  size x size x features
#  features are 
#   "how much halite on the board", 
#   "where are ships",
#   "how full are ships",
#   "where are bases",

# could add features around how many turns are left
# how much halite players have on the bank etc

def world_feature(board):
    size = board.configuration.size
    me = board.current_player
    
    ships = np.zeros((1, size, size))
    ship_cargo = np.zeros((1, size, size))
    bases = np.zeros((1, size, size))

    map_halite = np.array(board.observation['halite']).reshape(1, size, size)/1000

    for iid, ship in board.ships.items():
        ships[0, ship.position[1], ship.position[0]] = 1 if ship.player_id == me.id else -1
        ship_cargo[0, ship.position[1], ship.position[0]] = ship.halite/1000

    for iid, yard in board.shipyards.items():
        bases[0, yard.position[1], yard.position[0]] = 1 if yard.player_id == me.id else -1
        
    return np.concatenate([
        map_halite, 
        ships, 
        ship_cargo, 
        bases
    ], axis=0)


#As example take the first frame of the game
sample_obs = env.state[0].observation
board = Board(sample_obs, env.configuration)

feature = world_feature(board)
feature

array([[[ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
        [ 1.825,  2.281,  0.   ,  0.837,  0.   ,  2.281,  1.825],
        [ 0.664,  0.247,  0.   ,  0.058,  0.   ,  0.247,  0.664],
        [ 0.   ,  0.   ,  0.026,  0.058,  0.026,  0.   ,  0.   ],
        [ 0.664,  0.247,  0.   ,  0.058,  0.   ,  0.247,  0.664],
        [ 1.825,  2.281,  0.   ,  0.837,  0.   ,  2.281,  1.825],
        [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ]],

       [[ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
        [ 0.   , -1.   ,  0.   ,  0.   ,  0.   , -1.   ,  0.   ],
        [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
        [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
        [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
        [ 0.   ,  1.   ,  0.   ,  0.   ,  0.   , -1.   ,  0.   ],
        [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ]],

       [[ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
      

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
#import torchvision.transforms as T

class DQN(nn.Module):

    def __init__(self, h, w, outputs):
        super(DQN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=5, stride=2)
        self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=5, stride=2)
        self.bn2 = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d(32, 32, kernel_size=5, stride=2)
        self.bn3 = nn.BatchNorm2d(32)

        # Number of Linear input connections depends on output of conv2d layers
        # and therefore the input image size, so compute it.
        def conv2d_size_out(size, kernel_size = 5, stride = 2):
            return (size - (kernel_size - 1) - 1) // stride  + 1
        convw = conv2d_size_out(conv2d_size_out(conv2d_size_out(w)))
        convh = conv2d_size_out(conv2d_size_out(conv2d_size_out(h)))
        linear_input_size = convw * convh * 32
        self.head = nn.Linear(linear_input_size, outputs)

    # Called with either one element to determine next action, or a batch
    # during optimization. Returns tensor([[left0exp,right0exp]...]).
    def forward(self, x):
        x = x
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        return self.head(x.view(x.size(0), -1))

In [ ]:
#Now we define an agent consisting of a neural network
import random
import torch
import torch.nn as nn

torch.set_printoptions(profile="short")


#Simple network with three conv layers and a linear output
#We will interpret the output as a (n_actions, size, size) array and 
#do the .argmax over the actions to find the move we would take for every location on the board, if there was a ship

#You want to change this to your liking e.g.
#https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html#q-network
#or search for various pytorch DQN implementations
class SmallModel(nn.Module):
    def __init__(self, input_channels, num_actions):
        super(SmallModel, self).__init__()
        self.input_channels = input_channels
        self.num_actions = num_actions
        
        self.network = nn.Sequential(
            nn.Conv2d(
                in_channels=input_channels,
                out_channels=16,
                kernel_size=(3, 3),
                stride=1,
                padding=3, #use this padding to give each pixel more vision
                padding_mode='circular'
            ),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Conv2d(
                in_channels=16,
                out_channels=16,
                kernel_size=(3, 3),
                stride=1,
                padding=0, #this will make the padded first layer smaller again
                padding_mode='circular'
            ),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Conv2d(
                in_channels=16,
                out_channels=16,
                kernel_size=(3, 3),
                stride=1,
                padding=0, #this will make the padded first layer smaller again
                padding_mode='circular'
            ),
            nn.ReLU(),
            nn.BatchNorm2d(16),
        )
        
        self.output = nn.Linear(BOARD_SIZE*BOARD_SIZE*16, BOARD_SIZE*BOARD_SIZE*len(ACTIONS))
        
    def forward(self, features):
        x = self.network(features)
        x = x.view(features.shape[0], -1) #flatten
        x = self.output(x) #pass through linear layer
        
        return x.reshape(features.shape[0], self.num_actions, BOARD_SIZE, BOARD_SIZE)
                          

model = SmallModel(
    input_channels=4, #needs to be equal to the number of feature channels we have
    num_actions=len(ACTIONS)
)

target_model = SmallModel(
    input_channels=4, #needs to be equal to the number of feature channels we have
    num_actions=len(ACTIONS)
)

#predicting the feature from the cell above
feature_tensor = torch.from_numpy(feature).float().unsqueeze(0)
prediction = model(feature_tensor) 
print(prediction.shape) 
# Shape: (1, 5, 6, 6) = (batch_size, n_actions, y, x)
# move would then be model(feature_tensor).argmax(dim=1)
# to select the Q-Values for a specific location you can do prediction[0, :, y, x]  where y,x will be the coordinates of our ships
prediction

torch.Size([1, 5, 7, 7])


tensor([[[[ 0.28,  0.06,  0.39, -0.16,  0.80,  0.26, -0.74],
          [ 0.44, -0.72, -1.11, -0.05,  0.13,  0.96, -1.42],
          [ 0.05, -0.60,  0.94, -0.37,  0.25,  0.06, -0.13],
          [ 0.05, -0.45,  0.54, -0.97,  0.72, -1.07,  0.05],
          [-0.12, -0.10, -0.12,  0.38,  0.17, -1.49,  0.02],
          [ 0.33,  0.91, -0.77, -0.13,  0.96,  0.39, -0.22],
          [-0.36, -0.98, -0.12,  0.05,  0.11,  0.61,  0.86]],

         [[ 0.60, -0.02,  1.23,  0.78, -0.14, -0.23, -0.50],
          [ 0.63,  0.05, -0.91,  0.19, -0.06,  0.40,  0.06],
          [ 0.98,  0.35, -0.08,  0.27,  0.30, -0.94,  0.10],
          [-0.02, -0.70, -0.31,  0.02,  0.40, -0.38, -0.48],
          [-0.66, -0.79,  0.78, -1.07, -1.07, -0.06,  0.05],
          [ 0.61,  0.14,  0.17,  0.38,  0.93,  0.67, -0.25],
          [ 0.48,  0.20,  0.43, -0.71,  1.17, -1.04,  1.01]],

         [[ 0.04, -0.17,  0.59,  0.24, -0.66,  0.03, -0.54],
          [-0.81, -0.35,  0.20, -1.13,  0.46, -0.14, -0.11],
          [-0.40, -0

In [ ]:
def make_move(model, obs, configuration, EPSILON):
    size = configuration.size
    board = Board(obs, configuration)
    me = board.current_player


    #if we do not have ships but a shipyard build 1 ship
    if len(me.ships)==0 and len(me.shipyards)>0:
        me.shipyards[0].next_action = ShipyardAction.SPAWN

    #if we have no shipyard build one
    state = world_feature(board).astype(np.float32)
    state_tensor = torch.from_numpy(state).unsqueeze(0)
    print(state_tensor)
    action_indices = model(state_tensor).detach().numpy().argmax(1).squeeze()
    random_indices = np.random.choice(range(5), (size, size))
    actions = np.zeros((size, size))-1
    

    for ship in me.ships: 
        if len(me.shipyards)==0:
            action_index = -1
            ship.next_action = ShipAction.CONVERT #in our toy example we handle this manually
        else:
            if random.random() < EPSILON:
                action_index = random_indices[ship.position[1], ship.position[0]]
            else:
                action_index = action_indices[ship.position[1], ship.position[0]]
            
            ship.next_action = ACTIONS[action_index]
            
        actions[ship.position[1], ship.position[0]] = action_index
            
    return me.next_actions, state, actions

make_move(model, sample_obs, env.configuration, 0.0)   

tensor([[[[ 0.00,  0.00,  0.00,  0.00,  0.00,  0.00,  0.00],
          [ 1.83,  2.28,  0.00,  0.84,  0.00,  2.28,  1.83],
          [ 0.66,  0.25,  0.00,  0.06,  0.00,  0.25,  0.66],
          [ 0.00,  0.00,  0.03,  0.06,  0.03,  0.00,  0.00],
          [ 0.66,  0.25,  0.00,  0.06,  0.00,  0.25,  0.66],
          [ 1.83,  2.28,  0.00,  0.84,  0.00,  2.28,  1.83],
          [ 0.00,  0.00,  0.00,  0.00,  0.00,  0.00,  0.00]],

         [[ 0.00,  0.00,  0.00,  0.00,  0.00,  0.00,  0.00],
          [ 0.00, -1.00,  0.00,  0.00,  0.00, -1.00,  0.00],
          [ 0.00,  0.00,  0.00,  0.00,  0.00,  0.00,  0.00],
          [ 0.00,  0.00,  0.00,  0.00,  0.00,  0.00,  0.00],
          [ 0.00,  0.00,  0.00,  0.00,  0.00,  0.00,  0.00],
          [ 0.00,  1.00,  0.00,  0.00,  0.00, -1.00,  0.00],
          [ 0.00,  0.00,  0.00,  0.00,  0.00,  0.00,  0.00]],

         [[ 0.00,  0.00,  0.00,  0.00,  0.00,  0.00,  0.00],
          [ 0.00,  0.00,  0.00,  0.00,  0.00,  0.00,  0.00],
          [ 0.00,  0

({'0-1': 'CONVERT'},
 array([[[ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
         [ 1.825,  2.281,  0.   ,  0.837,  0.   ,  2.281,  1.825],
         [ 0.664,  0.247,  0.   ,  0.058,  0.   ,  0.247,  0.664],
         [ 0.   ,  0.   ,  0.026,  0.058,  0.026,  0.   ,  0.   ],
         [ 0.664,  0.247,  0.   ,  0.058,  0.   ,  0.247,  0.664],
         [ 1.825,  2.281,  0.   ,  0.837,  0.   ,  2.281,  1.825],
         [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ]],
 
        [[ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
         [ 0.   , -1.   ,  0.   ,  0.   ,  0.   , -1.   ,  0.   ],
         [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
         [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
         [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
         [ 0.   ,  1.   ,  0.   ,  0.   ,  0.   , -1.   ,  0.   ],
         [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ]],
 
        [[ 0.   ,  0.   ,  0.   ,  

In [ ]:
import torch.optim as optim
optimizer = optim.Adam(model.parameters(), lr=1e-4)
model

SmallModel(
  (network): Sequential(
    (0): Conv2d(4, 16, kernel_size=(3, 3), stride=(1, 1), padding=(3, 3), padding_mode=circular)
    (1): ReLU()
    (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding_mode=circular)
    (4): ReLU()
    (5): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding_mode=circular)
    (7): ReLU()
    (8): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (output): Linear(in_features=784, out_features=245, bias=True)
)

In [ ]:
#Taken and slightly modified from here: https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html#replay-memory

class ReplayMemory(object):

    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    def push(self, *args):
        """Saves a transition."""
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        
        self.memory[self.position] = args[0]
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [ ]:
from collections import defaultdict
import torch.nn.functional as F
import copy

EPSILON = 1.0 
EPSILON_DECAY = 0.998
TRAINING_ITERATIONS = 2300
EPOCHS = 1
REPLACE_TARGET_INTERVAL = 10
LEARNING_RATE = 0.1
REPLAY_CAPACITY = 100000
WARM_START_SAMPLES = 32*20
BATCH_SIZE = 32
GAMMA = 0.99
PRINT_INTERVAL = 100

running_avg_reward = []
episode_rewards = []

memory = ReplayMemory(REPLAY_CAPACITY)

#A bunch of stuff is happening here
#We inialize the environment, set random seeds and define some variables we need for storing all the necessary information
#We run one full game with 4 times the same agent (initially making random moves EPSILON=1)
#We store all the data in the replay buffer
#After the last round we need to properly set the done flag, calculate the last rewards and the next states for everything
#We then store this in the replay buffer
#If we have enough samples, we run EPOCHS times the training loop, random sampling data and training the network with it (r+GAMMA*target_q)

for episode in range(TRAINING_ITERATIONS+1): #+1 so its inclusive and we print a statement at the end
    print(f'{episode} - {round(EPSILON,3)} - {len(memory)}', end='\r')
    _ = env.reset(num_agents=NUM_AGENTS)

    #When we call env.reset it will set the random seeds of both python and numpy to our fixed value
    #We want to do some real random exploration though, otherwise we will always end up with the same game
    seed_time = int(time.time()*1000)%1000000000
    np.random.seed(seed_time)
    seed(seed_time)
    size = env.configuration.size


    player2states = defaultdict(list)
    player2actions = defaultdict(list)

    player2halite = defaultdict(list)
    player2rewards = defaultdict(list)

    player2dones = defaultdict(list)

    #The gist of this loop is copied from Tom Van de Wiele's answer here: https://www.kaggle.com/c/halite/discussion/144844
    while not env.done:
        observation = env.state[0].observation
        player_mapped_actions = []
        for active_id in range(NUM_AGENTS):
            agent_status = env.state[active_id].status
            if agent_status == 'ACTIVE':
                player_obs = env.state[0].observation.players[active_id]
                observation['player'] = active_id
                engine_commands, state, actions = make_move(model, observation, env.configuration, EPSILON)

                player2states[active_id].append(state)
                player2actions[active_id].append(actions)

                #in the first round there was no previous reward, we will use 5000 so the diff is 0 and drop it later in post processing
                #one big thing is that we only generate one reward per frame, probably we should generate a (size, size) reward array to 
                #properly attribute the rewards to the ships if we have a multi ship scenario later
                prev_reward = 5000 if len(player2halite[active_id]) == 0 else player2halite[active_id][-1]
                reward = player_obs[0] - prev_reward
                reward = reward if reward > 0 else 0
                player2rewards[active_id].append(reward) 
                player2halite[active_id].append(player_obs[0]) 

                player2dones[active_id].append(env.done)

                player_mapped_actions.append(engine_commands)
            else:
                player_mapped_actions.append({})
        env.step(player_mapped_actions)


    #Postprocessing:
    #We need to build (state(t), actions(t), reward(t+1), state(t+1), dones(t+1)) tuples
    #After the env finished we want to set the last done to true
    #We want to add the last reward and remove the reward t=0 (since we always need reward(t+1))
    for active_id in range(NUM_AGENTS):
        player_obs = env.state[0].observation.players[active_id]
        player2dones[active_id][-1] = True #the main loop does not get called again when the env is done so we set it manually

        prev_reward = player2halite[active_id][-1]
        reward = player_obs[0] - prev_reward
        reward = reward if reward > 0 else 0
        player2rewards[active_id].append(reward) #append reward t+1
        player2rewards[active_id].pop(0) #remove reward t=0

        #For debugging: Make sure we have the same number of samples everywhere
        #print(len(player2states[active_id]), len(player2actions[active_id]),len(player2rewards[active_id]),len(player2dones[active_id]),)
        #Look at your rewards and compare with the replay below whether the reward matches the games that you see
        #print(player2rewards[active_id])


        states = player2states[active_id]
        next_states = [x for x in states]
        next_states = next_states[1:] + next_states[-1:]

        for state, action, reward, next_state, done in zip(states, player2actions[active_id], player2rewards[active_id], next_states, player2dones[active_id]):
            memory.push((state, action, reward, next_state, done))


    episode_rewards.append(np.array([x for y in player2rewards.values() for x in y]))

        
        
    running_avg_reward.append(episode_rewards[-1].sum()/episode_rewards[-1].shape)
    if episode % PRINT_INTERVAL == 0:
        episode_rewards = np.concatenate(episode_rewards)
        print(f'ep:{episode}, '
               f'mem_size:{len(memory)}, '
               f'rew:{episode_rewards.sum()}, '
               f'avg:{round(episode_rewards.sum()/episode_rewards.shape[0], 3)}, '
               f'eps:{round(EPSILON, 2)}, '
               f'running_avg_rew:{round(np.mean(running_avg_reward), 3)}'
              )
        episode_rewards = []
    
            
            
    if not len(memory)>WARM_START_SAMPLES:
        continue
        
    for epoch in range(EPOCHS):
        sample = memory.sample(BATCH_SIZE)

        # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for detailed explanation). 
        # This converts [(state1, action1, reward1, next_state1, done1), (state2, action2, reward2, next_state2, done2)]
        # To:[(state1, state2), (action1, action2), (reward1, reward2), (next_state1, next_state2), (done1, done2)]
        states, actions, rewards, next_states, dones = list(zip(*sample))

        states = np.array(states)
        actions = np.array(actions)
        rewards = np.array(rewards)
        next_states = np.array(next_states)
        dones = np.array(dones)

        #Its not a bad idea to check shapes again: states.shape, next_states.shape, actions.shape, rewards.shape, dones.shape
        states = torch.from_numpy(states)
        next_states = torch.from_numpy(next_states)
        actions = torch.from_numpy(actions).long()
        rewards = torch.from_numpy(rewards)
        dones = torch.from_numpy(dones)

        #our actions are a (size, size) array, we want to select all fields that are not -1 since this is where a ship was that took an action
        #if we had mutliple ships then batch would contain the same frame multiple times with different x and y coordinates
        batch, xs, ys = np.where(actions>-1)

        taken_actions = actions[batch, xs, ys].unsqueeze(-1)

        #We will train multiple epochs, here you would ideally want to sample from a replay buffer

        current_qs = model(states)[batch, :, xs, ys].gather(1, taken_actions)
        next_qs = target_model(next_states).detach().max(1)[0][batch, xs, ys]

        # target_q = reward + 0.99 * next_state_max_q * (1 - done)
        target_qs = rewards[batch] + GAMMA * next_qs * ~dones[batch]
        loss = F.smooth_l1_loss(current_qs.squeeze(), target_qs.detach())
        #if we turn this on we will see that the loss is actually not decreasing very much from epoch to epoch
        #print(target_qs.shape, current_qs.shape, loss.mean(), end='\n')
        optimizer.zero_grad()
        loss.mean().backward()
        for param in model.parameters():
            param.grad.data.clamp_(-1, 1)
        optimizer.step()

    if episode and episode % REPLACE_TARGET_INTERVAL:
        target_model = copy.deepcopy(model)
        
    EPSILON *= EPSILON_DECAY 

NameError: name 'make_move' is not defined

In [ ]:
env = make('halite', configuration={"randomSeed": 1, "episodeSteps": TURNS, "size": BOARD_SIZE}, debug=True)
_ = env.reset(num_agents=NUM_AGENTS)

while not env.done:
    observation = env.state[0].observation
    player_mapped_actions = []
    for active_id in range(NUM_AGENTS):
        agent_status = env.state[active_id].status
        if agent_status == 'ACTIVE':
            player_obs = env.state[0].observation.players[active_id]
            observation['player'] = active_id
            engine_commands, state, actions = make_move(model, observation, env.configuration, 0.0)

            player_mapped_actions.append(engine_commands)
        else:
            player_mapped_actions.append({})
    env.step(player_mapped_actions)
    
    
env.render(mode='ipython', width=800, height=600)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x7056 and 784x245)